In [12]:
# Reading in packages
import pandas as pd
from SQLCode import DatabaseConnection
from SQLCode import DatabaseCredentials as DBC
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [14]:
# Opening connection
creds = DBC.DataBaseCredentials()
conn = DatabaseConnection.sql_connection(creds.server, creds.database, creds.user, creds.password)
connection = conn.open()
cursor = connection.cursor()

In [16]:
# Getting the live_feed data
live_feed = pd.read_sql_query("select * from live_feed", connection)

In [18]:
# Getting the seasons data
seasons = pd.read_sql_query("select * from schedules", connection)

In [20]:
# Filtering to regular seasons games and 20102011 onwards (when live data started)
seasonsFiltered = seasons[seasons['seasonID'] >= 20102011]
seasonsFiltered = seasonsFiltered[seasonsFiltered['gameType'] == 'R']

In [22]:
# Getting and filtering the raw data
rawData = pd.merge(live_feed,seasons, how='right',on='gameID')
rawData = rawData[rawData['seasonID'] >= 20102011]
rawData = rawData[rawData['gameType'] == 'R']

### Creating Initial Data Sets

In [24]:
# Filtering to the rows with the required data (goals and shots)
statsRaw = rawData[((rawData['eventTypeID'] == 'GOAL') & (rawData['playerType'] == 'Scorer')) |
               ((rawData['eventTypeID'] == 'SHOT') & (rawData['playerType'] == 'Shooter'))]

In [26]:
# Getting counts of the "for" stats.  
statsFor = pd.DataFrame(statsRaw.groupby(['gameID','teamID','eventTypeID']).size()).reset_index()
statsFor = statsFor.rename({0:'count'}, axis=1)

In [28]:
# Pivoting so each stat gets its own column
statsFor = pd.pivot_table(data=statsFor, values='count',index=['gameID','teamID'], columns='eventTypeID').reset_index()

In [30]:
# Defing the stats "against" table
statsAgainst = statsRaw[['gameID','eventTypeID']]

In [32]:
# Getting the "against" team
statsAgainst['teamID'] = statsRaw.apply(lambda row: row['homeTeamID'] if row['teamID'] != row['homeTeamID'] else row['awayTeamID'],axis=1)

<ipython-input-32-836c260b681f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statsAgainst['teamID'] = statsRaw.apply(lambda row: row['homeTeamID'] if row['teamID'] != row['homeTeamID'] else row['awayTeamID'],axis=1)


In [34]:
# Getting the stats counts
statsAgainst = pd.DataFrame(statsAgainst.groupby(['gameID','teamID','eventTypeID']).size()).reset_index()
statsAgainst = statsAgainst.rename({0:'count'}, axis=1)

In [36]:
# Pivoting so each stat gets its own column
statsAgainst = pd.pivot_table(data=statsAgainst, values='count',index=['gameID','teamID'], columns='eventTypeID').reset_index()

### Create complete schedule table

In [38]:
# Expanding the schedule into a row for each game an individual team played
seasonsExpanded = pd.concat([seasonsFiltered[['seasonID','gameID','gameType','homeTeamID']].rename({'homeTeamID':'teamID'},axis=1),
          seasonsFiltered[['seasonID','gameID','gameType','awayTeamID']].rename({'awayTeamID':'teamID'},axis=1)])

### Extending data set for every game

In [40]:
# Merging the stats for and against into a single table.  
# Notice the left join on seasonsExpanded to ensure we aren't losing any data
stats = pd.merge(seasonsExpanded, 
                 statsFor, 
                 how='left', 
                 left_on=['gameID','teamID'],
                 right_on=['gameID','teamID'])
stats = stats.rename({'GOAL':'goalsFor','SHOT':'shotsFor'},axis=1)
stats = pd.merge(stats , 
                 statsAgainst, 
                 how='left', 
                 left_on=['gameID','teamID'],
                 right_on=['gameID','teamID'])
stats = stats.rename({'GOAL':'goalsAgainst','SHOT':'shotsAgainst'},axis=1)

In [42]:
# Setting no stats (i.e. NA) to 0
stats['goalsFor'] = stats['goalsFor'].apply(lambda x: 0 if pd.isna(x) else x)
stats['goalsAgainst'] = stats['goalsAgainst'].apply(lambda x: 0 if pd.isna(x) else x)
stats['shotsAgainst'] = stats['shotsAgainst'].apply(lambda x: 0 if pd.isna(x) else x)
stats['shotsFor'] = stats['shotsFor'].apply(lambda x: 0 if pd.isna(x) else x)

In [44]:
# Sorting the values
stats = stats.sort_values(['seasonID','gameID'])

# Grabbing this now to use in "Extracting the Game Outcome"
statsSimple = stats

### Creating the Statistics

In [46]:
# Creating a game number for every game per team
stats['gameNumber'] = stats.groupby(['seasonID','teamID']).cumcount()+1

In [48]:
# Getting the rolling totals for each stat
stats['goalsAgainstTotal'] = stats.groupby(['seasonID','teamID'])['goalsAgainst'].cumsum()
stats['goalsForTotal'] = stats.groupby(['seasonID','teamID'])['goalsFor'].cumsum()
stats['shotsForTotal'] = stats.groupby(['seasonID','teamID'])['shotsFor'].cumsum()
stats['shotsAgainstTotal'] = stats.groupby(['seasonID','teamID'])['shotsAgainst'].cumsum()

In [50]:
# Creating a game number for each team/season
stats['gameNumber'] = stats.groupby(['seasonID','teamID']).cumcount()+1

In [52]:
# Getting the rolling totals per game for each stat
stats['goalsAgainstPerGame'] = stats['goalsForTotal']/stats['gameNumber']
stats['goalsForPerGame'] = stats['goalsAgainstTotal']/stats['gameNumber']
stats['shotsAgainstPerGame'] = stats['shotsAgainstTotal']/stats['gameNumber']
stats['shotsForPerGame'] = stats['shotsForTotal']/stats['gameNumber']

In [54]:
# Getting the rolling differentials
stats['goalDifferential'] = stats['goalsForTotal'] - stats['goalsAgainstTotal']
stats['shotDifferential'] = stats['shotsForTotal'] - stats['shotsAgainstTotal']

In [55]:
# Determing whether a team won/lossed and creating a rolling win percentage
stats['winLoss'] = stats.apply(lambda row: 1 if row['goalsFor'] > row['goalsAgainst'] else 0,axis=1)
stats['winLossTotal'] = stats.groupby(['seasonID','teamID'])['winLoss'].cumsum()
stats['winningPercentage'] = stats['winLossTotal']/stats['gameNumber']

In [56]:
statsComplete = stats.drop(['goalsFor',
                            'goalsAgainst', 
                            'shotsAgainst', 
                            'shotsFor',
                            'gameNumber',
                            'goalsForTotal',
                            'goalsAgainstTotal',
                            'shotsAgainstTotal',
                            'shotsForTotal',
                            'winLoss',
                            'winLossTotal'], axis=1)

### Adding Q Values

In [66]:
# Getting the seasons data
boxscores = pd.read_sql_query("select gameID, teamID,playerID from box_scores where timeOnIce is not null and scratched=0", connection)

In [67]:
# Merging boxscores with seasons to get the seasonID
boxscores = pd.merge(seasonsFiltered[['seasonID','gameID']], boxscores, how='inner')

In [68]:
# Sorting to make sure the row count is correct
boxscores = boxscores.sort_values(['seasonID','gameID'])

In [ ]:
# # Getting the previous game for each game/player combo
previousGame = []
for index, row in boxscores.iterrows():
    if index % 10000 == 0:
        print(int((index/len(boxscores))*100),'%')
    preGame = boxscores[(boxscores['gameID'] < row['gameID']) & (boxscores['playerID'] == row['playerID'])]['gameID'].values
    if len(preGame) == 0:
        previousGame.append(np.nan)
    else:
        previousGame.append(max(preGame))

0 %
2 %
4 %
6 %
8 %
10 %
12 %
14 %
16 %
18 %
20 %
22 %
24 %
27 %
29 %
31 %
33 %
35 %
37 %
39 %
41 %
43 %
45 %
47 %
49 %
51 %
54 %
56 %
58 %
60 %
62 %


In [ ]:
seasonsList = seasonsFiltered['seasonID'].unique()
previousSeasons = []
for index, row in boxscores.iterrows():
    if index % 10000 == 0:
        print(int((index/len(boxscores))*100),'%')
    season = seasonsList[seasonsList < row['seasonID']]
    if len(season) == 0:
        previousSeasons.append(np.nan)
    else:
        previousSeasons.append(max(season))
    
# #     break
    

In [ ]:
boxscores['previousSeason'] = previousSeasons
boxscores['previousSeason'] = boxscores['previousSeason'].fillna(20092010)
boxscores['previousSeason'] = boxscores['previousSeason'].astype(int)

In [ ]:
# Getting the game number if the player played in the game
boxscores['gameNumber'] = boxscores.groupby(['seasonID','playerID']).cumcount()+1

In [64]:
# boxscores = pd.read_csv('boxscores.csv',index_col=0)
# boxscores.to_csv('boxscores.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'boxscores.csv'

In [65]:
# Reading in the q values
qValues = pd.read_csv('deep_rl_results_all_games.csv',index_col=0)

C:\Users\Aidan\anaconda3\envs\tf\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
# Summing up the values by game
qValues = qValues[['gameID',
                   'playerID',
                   'value']].groupby(['gameID',
                                         'playerID']).sum('value').reset_index()

In [ ]:
# Merging with boxscores
qValues = pd.merge(boxscores, qValues, on=['gameID','playerID'],how='left')

In [ ]:
# Filling NA values (i.e. games where a player didn't show up)
qValues['value'] = qValues['value'].fillna(0)

In [ ]:
qValues = qValues.sort_values(['seasonID','gameID'])

In [ ]:
# Summing up the q values by player/season/game
qValues['cumulativeValue'] = qValues.groupby(['seasonID','playerID'])['value'].cumsum(axis=0)

In [ ]:
# Getting the q value per game
qValues['averageValue'] = qValues['cumulativeValue']/qValues['gameNumber']

In [ ]:
qValues[qValues['playerID'] == 8478402]

In [ ]:
# Getting the average q value per game by season
qSeasonValues = qValues[['seasonID','averageValue']].groupby('seasonID').mean().reset_index()
# Renaming the column
qSeasonValues = qSeasonValues.rename({'averageValue':'averageValueSeason'},axis=1)

In [ ]:
# Merging boxscores with qValues to get the qValues up to the last game
boxScoresComplete = pd.merge(boxscores, 
         qValues[['gameID','playerID','averageValue']],
         left_on=['previousGameID','playerID'],
         right_on=['gameID','playerID'], 
         how='left',
         suffixes=('', '_y'))

In [ ]:
# Getting the qValues by season
boxScoresComplete = pd.merge(boxScoresComplete,
         qSeasonValues, 
         left_on='previousSeason',
         right_on='seasonID',
         how='inner',
         suffixes=('', '_y'))

In [ ]:
# Dropping useless columns
boxScoresComplete = boxScoresComplete.drop(['gameID_y','seasonID_y'],axis=1)

In [ ]:
# setting the averageValue 
boxScoresComplete['averageValue'] = np.where(boxScoresComplete['averageValue'].isna(),boxScoresComplete['averageValueSeason'],boxScoresComplete['averageValue'])

In [ ]:
boxScoresComplete = boxScoresComplete[['seasonID',
                   'gameID',
                   'teamID',
                   'averageValue']].groupby(['seasonID','gameID','teamID']).sum().reset_index()

In [ ]:
statsComplete = pd.merge(statsComplete, boxScoresComplete)

In [ ]:
statsComplete

### Extracing the game Outcome

In [ ]:
# Getting goals for/against columns
gameOutcome = pd.merge(seasonsFiltered,
         statsSimple[['seasonID','gameID','teamID','goalsFor','goalsAgainst']], 
         how='inner',
         left_on=['seasonID','gameID','homeTeamID'],
         right_on=['seasonID','gameID','teamID'])

In [ ]:
# Determing if the home team won
gameOutcome['homeTeamWin'] = gameOutcome.apply(lambda row: 1 if row['goalsFor'] > row['goalsAgainst'] else 0,axis=1)

In [ ]:
# Getting rid of useless columns
gameOutcome = gameOutcome.drop(['gameType', 
                  'gameDate', 
                  'homeTeamID',
                  'awayTeamID',
                  'teamID','seasonID','goalsFor','goalsAgainst'],axis=1)

### Creating Model Input

In [ ]:
# Determining if the team is the home or away, this is needed to join properly again on seasonsFiltered below
statsComplete = pd.merge(statsComplete,seasonsFiltered[['gameID','homeTeamID']], 
                 how='left',
                 left_on=['gameID','teamID'],
                 right_on=['gameID','homeTeamID'])
statsComplete['isHome'] = np.where(pd.isna(statsComplete['homeTeamID']), 0, 1)
statsComplete = statsComplete.drop(['homeTeamID'],axis=1)

In [ ]:
# Determing the game number for each time by season
statsComplete = statsComplete.sort_values(['seasonID','gameID'])
statsComplete['gameNumber'] = statsComplete.groupby(['seasonID','teamID','isHome']).cumcount()+1
statsComplete['gameNumber'] += 1

In [ ]:
# Sorting and then getting the home and away teams game numbers (used to join on below)
seasonsFiltered = seasonsFiltered.sort_values(['seasonID','gameID'])
seasonsFiltered['homeTeamGameNumber'] = seasonsFiltered.groupby(['seasonID','homeTeamID']).cumcount()+1
seasonsFiltered['awayTeamGameNumber'] = seasonsFiltered.groupby(['seasonID','awayTeamID']).cumcount()+1

In [ ]:
# Creating the model input with the home team data
modelInput = pd.merge(seasonsFiltered, 
            statsComplete[statsComplete['isHome'] == 1], 
            how='inner', 
            left_on=['seasonID', 'homeTeamID', 'homeTeamGameNumber'],
            right_on=['seasonID','teamID','gameNumber'])

In [ ]:
# Dropping extra columns
modelInput = modelInput.drop(['gameID_x', 
                 'gameType_x',
                 'homeTeamGameNumber',
                 'teamID',
                 'gameNumber'],
                axis=1)

In [ ]:
# Renaming some columns to keep things clean
modelInput = modelInput.rename({"gameID_y":"gameID", "gameType_y":"gameType"}, axis=1)

In [ ]:
# adding the away team data to the model input
modelInput = pd.merge(modelInput, 
            statsComplete[statsComplete['isHome'] == 0], 
            how='inner', 
            left_on=['seasonID', 'awayTeamID', 'awayTeamGameNumber'],
            right_on=['seasonID','teamID','gameNumber'], suffixes=('Home','Away'))

In [ ]:
pd.set_option("display.max_columns", None)
modelInput

In [ ]:
# Dropping columns that are no longer needed
modelInput = modelInput.drop(['awayTeamGameNumber',
                              'gameIDHome',
                              'gameTypeHome',
                              'teamID',
                              'gameNumber',
                              'isHomeAway'],axis=1)

In [ ]:
# Renaming these to get rid of the suffix that was put on it on the last join
modelInput = modelInput.rename({'gameIDAway':'gameID','gameTypeAway':'gameType'},axis=1)

In [ ]:
modelInput = pd.merge(modelInput, gameOutcome, on='gameID')

## Creating the Model

### Normalizing the Data and creating train/test data

In [ ]:
# Creating the x and y data
x = modelInput.drop(['seasonID',
                     'gameDate',
                     'homeTeamID',
                     'awayTeamID',
                     'gameID',
                     'gameType',
#                      'shotsAgainstPerGameHome', 
#                      'shotsForPerGameHome', 
#                      'shotDifferentialHome',
#                      'shotsAgainstPerGameAway', 
#                      'shotsForPerGameAway',
#                      'shotDifferentialAway',
                     'homeTeamWin'],axis=1)
y = modelInput[['homeTeamWin']]

In [ ]:
# Normalizing the data
scaler = preprocessing.StandardScaler().fit(x)
xScaled = scaler.transform(x)

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(xScaled, y, test_size=0.2,random_state=109) # 70% training and 20% test

### Support Vector Machine

In [ ]:
clf = svm.SVC(kernel='linear',gamma='auto') # Linear Kernel

In [ ]:
clf.fit(xTrain, yTrain['homeTeamWin'].values)

In [ ]:
yPred = clf.predict(xTrain)
print("Train Accuracy:",metrics.accuracy_score(yTrain, yPred))
yPred = clf.predict(xTest)
print("Test Accuracy:",metrics.accuracy_score(yTest, yPred))